<font color='darkblue' size=18>Zonas Basicas de Salud con python</font><img src="img/Mercator-projection.jpg" align='right'><img src="img/python-logo.png" align='right'>

### <font color='darkblue' size=12> Comunidad de Madrid</font>

#### Vamos a usar python y alguna de sus librerías para crear una representación gráfica de datos sobre un mapa.

##### Para ello necesitamos:

    1. Un mapa base.
    2. Shape de cada área del mapa.
    3. Datos a representar de cada área/shape.
    4. Asociar los datos de ZBS a cada shape.
    5. Mapa con los datos.

#### En este caso, serán datos de situación del COVID-19 en las Zonas Básicas de Salud de la Comunidad de Madrid.
### Necesitamos datos:
- Los shapes de las Zonas Básicas de Salud que forman el mapa de la Comunidad de Madrid.
- Un dato a representar en el mapa. He escogido la Tasa de Incidencia Acumulada en los últimos 14 días. Esta serie de datos se llamará **TIA_14d**.

### Necesitamos librerías que nos permitan acceder a los datos, procesarlos y presentarlos de manera interactiva:
- Podemos usar datos en local ó acceder a ellos a traves de su url pública. En este caso será con ficheros locales. El ejemplo sin ficheros locales? **ZBS_Madrid_url.ipynb**
- Vamos a usar hvplot por su sencillez, está muy relacionado con pandas y permite acceder a holoviews y bokeh. Terminando con opciones interactivas.


<br><br>
### Empecemos importando las librerías que vamos a usar

In [ ]:
# # Las dos siguientes líneas de código nos permite usar el 100% del ancho de pantalla.
from IPython.display import display, HTML, Markdown
display(HTML("<style>.container { width:100% !important; }</style>"))


from IPython.core.interactiveshell import InteractiveShell  # Para mostrar más de una salida por celda en el notebook.

In [ ]:
import geopandas as gpd
import geoviews as gv
import cartopy.crs as ccrs
import pandas as pd
import numpy as np
import panel as pn
from bokeh.models import FixedTicker as bk_FixedTicker, HoverTool as bk_HoverTool
import holoviews as hv

gv.extension('bokeh')

<br><br>
## 1. Mapa Base.

    Será de Wikipedia. Hay otras opciones posibles de Mapa Base.

In [ ]:
Mapa_Base = gv.tile_sources.Wikipedia
Mapa_Base  # Ya tenemos Mapa Base y es interactivo!!

---
---
<br><br>
## 2. Colección de shapes de ZBS de la Comunidad de Madrid.


### - Cargamos en un GeoDataFrame los shapes y una columna extra pob_pad19. Los ficheros los proporciona la Comunidad de Madrid <img src="img/logo_comunidad.png" align='right' />
- La CAM proporciona un fichero zip con los shapes de las Zonas Básicas Sanitarias. [Datos de shapes](https://datos.comunidad.madrid/catalogo/dataset/b3d55e40-8263-4c0b-827d-2bb23b5e7bab/resource/f1837bd3-a835-4110-9bbf-fae06c99b56b/download/zonas_basicas_salud.zip)

[WEB Datos de Zonas Básicas de Madrid](https://datos.comunidad.madrid/catalogo/dataset/covid19_tia_zonas_basicas_salud)
- Podríamos acceder a estos ficheros directamente online ó descargarlo y usarlo como fichero local.
<br><br>
#### **PREVIAMENTE** he descargado esos ficheros a un subdirectorio llamado 'zonas_basicas_salud'
### Cargando los shapes

In [ ]:

gdf_CAM = gpd.read_file('zonas_basicas_salud') # Le he indicado el nombre del subdirectorio y geopandas se encarga de cargar lo necesario.

gdf_CAM_Mercator = gdf_CAM.to_crs(ccrs.GOOGLE_MERCATOR.proj4_init)  # Hay que adaptar las coordenadas de los shapes al estandar Mercator. Dejo los dos GeoDataFrame por si se quieren ver después.

In [ ]:
Mapa_ZBS = gv.Polygons(gdf_CAM_Mercator, vdims=['pob_pad19', 'zona_basic'], crs=ccrs.GOOGLE_MERCATOR)  
Mapa_ZBS   # Ya tenemos la capa de ZBS!!

### 2.1 Juntamos el Base y los Shapes de ZBS

In [ ]:
Mapa_Base * Mapa_ZBS  # Juntando Piezas.

----
----
<br><br>
## 3. Cargamos datos de COVID para cada ZBS

- La CAM proporciona un fichero csv con los datos COVID por Zona Básica Sanitaria. Se actualiza aproximadamente, una vez por semana, y suele ser los martes.  [Datos COVID-19 por Zona Básica Sanitaria](https://datos.comunidad.madrid/catalogo/dataset/b3d55e40-8263-4c0b-827d-2bb23b5e7bab/resource/43708c23-2b77-48fd-9986-fa97691a2d59/download/covid19_tia_zonas_basicas_salud_s.csv)

In [ ]:
# Cargamos los datos y los preparamos un poquito.

df_zonas_basicas = pd.read_csv('zonas_basicas_salud/covid19_tia_zonas_basicas_salud_s.csv', sep=';', encoding='iso8859_15', decimal=',', parse_dates=['fecha_informe'])

df_zonas_basicas.rename(columns={'tasa_incidencia_acumulada_ultimos_14dias':'TIA_14d'}, inplace=True)  # Cambio el nombre de la columna por comodidad posterior.

df_zonas_basicas['codigo_geo']=df_zonas_basicas['codigo_geometria'].str.strip()  # Los codigos vienen con espacios por detrás. Se los quito y aprovecho para cambiar el nombre.
df_zonas_basicas['TIA_14d']=df_zonas_basicas['TIA_14d'].astype('int')
df_zonas_basicas.drop(columns='codigo_geometria', inplace=True)  # Ya no es necesaria esta columna.

df_zonas_basicas.head(3)


<br><br>
### 3.1 Filtramos los últimos datos del fichero

In [ ]:
#  En el fichero vienen los datos semanales desde el 2 de julio. Ahora, queremos el último dato de cada ZBS. 

df_zbs_ultimo_dia = df_zonas_basicas.loc[(df_zonas_basicas.fecha_informe==df_zonas_basicas.fecha_informe.max()),['TIA_14d','codigo_geo','fecha_informe']]

df_zbs_ultimo_dia['fecha_informe']=df_zbs_ultimo_dia.fecha_informe.dt.strftime('%Y-%m-%d')

df_zbs_ultimo_dia.info(memory_usage='deep')

<br><br>
Vemos un DataFrame con 286 registros.

Uno por cada ZBS
Con 3 campos en cada registro.
- Codigo_geo:     Identifica a cada ZBS.
- fecha_informe:  fecha de los datos
- TIA-14:         Tasa de Incidencia Acumulada en los últimos 14 días.

In [ ]:
df_zbs_ultimo_dia.head(3)


----
----

<br><br>
## 4. Añadimos los datos de TIA_14d al GeoDataFrame

In [ ]:
gdf_CAM_COVID=gdf_CAM_Mercator.merge(df_zbs_ultimo_dia,  on='codigo_geo')
gdf_CAM_COVID.info(memory_usage='deep')

In [ ]:
Mapa_ZBS_COVID = gv.Polygons(gdf_CAM_COVID, vdims=['TIA_14d',('zona_basic', 'Zona Básica de Salud'), ('fecha_informe', 'Fecha del Dato')], crs=ccrs.GOOGLE_MERCATOR, hover_cols=['pob_pad19']).opts( line_color='grey', cmap='magma_r')
Mapa_ZBS_COVID

---
---
### 5. Mapa interactivo con los datos.

 - Juntamos Mapa Base y la capa Mapa_ZBS_COVID.
 - Tocamos algo las opciones para adaptar la visibilidad.

In [ ]:
df_zbs_ultimo_dia.info(memory_usage='deep')

In [ ]:
# Definimos un colormap a medida para la ocasión.
colormap_COVID = ['#006837' , '#86cb66', '#ffffbf', '#f98e52', '#a50026', '#000000']

Niveles_COVID = [0, 25, 50, 150, 250, 400, 430]

Colorbar_ticks = bk_FixedTicker(ticks=[  12.5, 25, 37.5, 50, 100, 150, 200, 250, 325, 400, 430])

Colorbar_labels =  {
            37.5: 'Riesgo Bajo',
            12.5: 'Nueva Normalidad',
            25:   '25',
            50:   '50',
            100: 'Riesgo Medio',
            150:  '150',
            200: 'Riesgo Alto',
            250:  '250',
            325: 'Riesgo Extremo',
            400:  '400',
            430: 'Riesgo IDA Extremo'
        }


# Mapa Base con Shapes de ZBS y con gradiente de color según TIA_14d

In [ ]:
Mapa_Base.opts(alpha=0.7, xaxis=None, yaxis=None) * \
Mapa_ZBS_COVID.opts( color='TIA_14d',  width=1000, xaxis=None, yaxis=None, height=900, tools=['hover'], alpha=0.75,
                    color_levels=Niveles_COVID, clim=(0,430), 
                    cmap= colormap_COVID, colorbar=True, colorbar_opts={
                    'major_label_overrides': Colorbar_labels,
                    'major_label_text_align': 'left', 'ticker': Colorbar_ticks
                    })

# Hacemos otro?

Fácil de adaptar, el tamaño, la transparencia ('alpha') de los shapes. 

In [ ]:
Mapa_ZBS_COVID.opts(alpha=0.9, width=700,height=630)

----
<br><br>
## Y cuando lo exportamos en formato html. Sigue interactivo!!

In [ ]:
gv.renderer('bokeh').save( Mapa_ZBS_COVID, 'ZBS_Madrid')  # aquí hemos creado el fichero ZBS_Madrid.html

<br><br>

## Lo hacemos más interactivo con opción de Vistas por Condición?

    - Por ejemplo: Filtrar las ZBS con TIA_14d mayor que un valor indicado. Es fácil.
    
    AVISO: Tarda un poquito en realizar el filtrado.

In [ ]:
# Creamos un pop-up a medida

hover_ZBS = bk_HoverTool(tooltips= [  ("Zona Básica Sanitaria  ", "  @zona_basic"), ("TIA-14 ", " @{TIA_14d}{0.0}"), ("Dia " , " @{fecha_informe}{%F}") ],
                         formatters={'@{fecha_informe}': 'datetime'} )



In [ ]:
# Creamos una función para reutilizarla

def Actualizar_polys(umbral):

    return gv.Polygons(gdf_CAM_COVID[gdf_CAM_COVID['TIA_14d'] > umbral], vdims=['TIA_14d', 'zona_basic','fecha_informe'], crs=ccrs.GOOGLE_MERCATOR).opts(clim=(0, 430), width=900, height=950, tools=[hover_ZBS], alpha=0.75,
                    color_levels=Niveles_COVID, line_color='grey',
                    cmap= colormap_COVID, colorbar=True, colorbar_opts={
                    'major_label_overrides': Colorbar_labels,
                    'major_label_text_align': 'left', 'ticker': Colorbar_ticks
                    })



wdg_IntSlider = pn.widgets.IntSlider(name='TIA_14d Mayor que ', start=0, end=int(gdf_CAM_COVID['TIA_14d'].max()-1))


pn.Row(
    wdg_IntSlider,
    gv.tile_sources.Wikipedia(alpha=0.6) * 
    gv.DynamicMap(pn.bind(Actualizar_polys, wdg_IntSlider))
)



In [ ]:
# Otro Semejante, igual de interactivo SIN Mapa Base


pn.Column( Markdown(f"## <br><br> Puedes filtrar el valor de TIA-14d."),
           Markdown(f"### Presentará las ZBS con TIA_14d Mayor de la seleccionada."),
           pn.Row(
                wdg_IntSlider,
                gv.DynamicMap(pn.bind(Actualizar_polys, wdg_IntSlider))
            )
        )

<br><br>
### Ahora creamos un GeoDataFrame con los datos históricos de las ZBS para todas las fechas desde el verano gdf_zbs_historico.

In [ ]:

gdf_zbs_historico = gdf_CAM.merge(df_zonas_basicas, on='codigo_geo')

gdf_zbs_historico.info(memory_usage='deep')


<br><br>
## Permitimos interactuar tanto por umbral de TIA_14d como la evolución en el tiempo de todas las ZBS.


In [ ]:
import holoviews as hv

def Actualizar_polys_fecha(umbral, fecha):
    
    return hv.Polygons(gdf_zbs_historico.loc[(gdf_zbs_historico['TIA_14d'] > umbral) & (gdf_zbs_historico.fecha_informe.dt.strftime("%F")==fecha)], vdims=['TIA_14d', 'zona_basic', 'fecha_informe']).opts(
                    clim=(0, 430), width=900, height=950, tools=[hover_ZBS], alpha=0.65, xaxis=None, yaxis=None,
                    color_levels=Niveles_COVID, line_color='grey',
                    cmap= colormap_COVID, colorbar=True, colorbar_opts={
                    'major_label_overrides': Colorbar_labels,
                    'major_label_text_align': 'left', 'ticker': Colorbar_ticks
                    }, bgcolor='lightgray')



fechas = sorted(gdf_zbs_historico.fecha_informe.dt.strftime("%F").unique())

wdg_IntSlider = pn.widgets.IntSlider(name='TIA_14d Mayor que ', start=0, end=int(gdf_zbs_historico['TIA_14d'].max()-1))
wdg_Slider = pn.widgets.DiscreteSlider(name='Dia ', options=fechas, value=fechas[-1])

pn.Row( pn.Column(
            Markdown(f"## <br><br> Puedes filtrar el valor de TIA-14d."),
            Markdown(f"### Presentará las ZBS con TIA_14d Mayor de la seleccionada."),
            Markdown(f"<br>## Se puede seleccionar otras fechas para ver el pasado y la evolución"),
    wdg_IntSlider, wdg_Slider),
    gv.DynamicMap(pn.bind(Actualizar_polys_fecha, wdg_IntSlider, wdg_Slider))
)

